# Interaction Prediction

*State of the Interactomes: an evaluation of molecular networks for generating biological insights.* Wright, SN et al. [https://doi.org/10.1101/2024.04.26.587073]  
This notebook contains code and results related to:
* Figure 5A-G

**Note: due to the size of network data sets utilized in this study, this notebook draws on summarized results rather than re-evaluating all networks directly.**


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from neteval.network_statistics import NetworkStats, load_network_names
from tqdm import tqdm
import os

In [2]:
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 7})
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 7})
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['hatch.linewidth'] = 0.5
plt.rcParams['xtick.major.width'] = 0.4
plt.rcParams['ytick.major.width'] = 0.4
plt.rcParams['xtick.minor.width'] = 0.3
plt.rcParams['ytick.minor.width'] = 0.3
plt.rcParams['legend.frameon'] = False
plt.rcParams['xtick.major.size'] = 3
plt.rcParams['ytick.major.size'] = 3
plt.rcParams['xtick.minor.size'] = 2
plt.rcParams['ytick.minor.size'] = 2
plt.rcParams['xtick.major.pad'] = 1
plt.rcParams['ytick.major.pad'] = 1
plt.rcParams['axes.labelpad'] = 1
plt.rcParams['patch.linewidth'] = 0.25


In [3]:
gitdir = os.path.expanduser('~/Git/Network_Evaluation_Tools/')
figdir = os.path.join(gitdir, 'StateOfTheInteractomes_Notebooks/Figures/')
evaldir = os.path.join(gitdir, 'Data/example_outputs/')
datadir = os.path.join(gitdir, 'Data/')
network_names = load_network_names(os.path.join(gitdir, 'StateOfTheInteractomes_Notebooks/Data/network_names_final.txt'))

## Example Analysis

With DIP, PID2, and Wan

In [4]:
l3_dir = os.path.join(evaldir, 'L3/')

In [5]:
with open(os.path.join(datadir, 'example_prefix_file.txt')) as f:
    prefixes = f.read().splitlines()

In [10]:
results = []
failed = []
failed_count = 0
for prefix in tqdm(prefixes):
    for i in range(1,11):
        for suffix in ['', 'corum_', 'panther_']:
            try:
                df = pd.read_csv(l3_dir + '{}.fold{}_L3_{}results.tsv'.format(prefix, i, suffix), sep='\t', index_col=0)
                df['Network']=prefix
                df['Fold']=i
                df['Type']=suffix
                results.append(df)
            except FileNotFoundError as e: 
                failed.append((prefix, i, suffix))
                failed_count +=1 
                
print('Succeeded: {}'.format(len(results)))
print('Failed: {}'.format(failed_count))
l3_df = pd.concat(results).reset_index()

100%|██████████| 3/3 [00:00<00:00, 24.60it/s]

Succeeded: 70
Failed: 20
